In [1]:
import pandas as pd
import numpy as np
import joblib
import pymysql

In [2]:
db_name = "gis_system"
db_host = "localhost"
db_username = "root"
db_password = ""

try:
    conn = pymysql.connect(host = db_host,
                           port = int(3306),
                           user = "root",
                           password = db_password,
                           db = db_name)
except e:
    print (e)
if conn:
    print ("connection successful")
else:
    print ("error")

connection successful


In [3]:
model = joblib.load('model/ModelClassifier.pkl')

In [4]:
df_period = pd.read_csv('dataset/period3_psch_ht.csv')
df_crop_recom_range = pd.read_csv('dataset/crop_recommendation_range.csv')

In [5]:
df_period = df_period.rename(columns={
    'rata_rata_humi': 'humidity',
    'Curah_Hujan_Rata': 'rainfall',
    'rata_rata_temp': 'temperature'
})

In [6]:
features = df_period[['temperature', 'humidity', 'rainfall']].copy()

In [7]:
predictions = model.predict(features)

df_period['predicted_usable'] = predictions

In [10]:
for index, row in df_period.iterrows():
    cursor = conn.cursor()
    sql = "SELECT * FROM predictions WHERE lat = %s AND lon = %s"
    cursor.execute(sql, (row['LAT'], row['LON']))
    result = cursor.fetchall()
    if len(result) > 0:
        sql = "UPDATE predictions SET provinsi = %s, rainfall = %s, sifat_hujan_rata = %s, temperature = %s, humidity = %s, predicted = %s WHERE lat = %s AND lon = %s"
        cursor.execute(sql, (row['Id Prov'], row['rainfall'], row['Sifat_Hujan_Rata'], row['temperature'], row['humidity'], row['predicted_usable'], row['LAT'], row['LON']))
    else:
        sql = "INSERT INTO predictions (lat, lon, provinsi, rainfall, sifat_hujan_rata, temperature, humidity, predicted) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, (row['LAT'], row['LON'], row['Id Prov'], row['rainfall'], row['Sifat_Hujan_Rata'], row['temperature'], row['humidity'], row['predicted_usable']))
    conn.commit()
    cursor.close()
    
for index, row in df_crop_recom_range.iterrows():
    cursor = conn.cursor()
    sql = "SELECT * FROM crop_recom_range WHERE label = %s"
    cursor.execute(sql, (row['label']))
    result = cursor.fetchall()
    if len(result) > 0:
        sql = "UPDATE crop_recom_range SET min_range_temperature = %s, max_range_temperature = %s, min_range_humidity = %s, max_range_humidity = %s, min_range_rainfall = %s, max_range_rainfall = %s WHERE label = %s"
        cursor.execute(sql, (row['min_range_temperature'], row['max_range_temperature'], row['min_range_humidity'], row['max_range_humidity'], row['min_range_rainfall'], row['max_range_rainfall'], row['label']))
    else:
        sql = "INSERT INTO crop_recom_range (label, min_range_temperature, max_range_temperature, min_range_humidity, max_range_humidity, min_range_rainfall, max_range_rainfall) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, (row['label'], row['min_range_temperature'], row['max_range_temperature'], row['min_range_humidity'], row['max_range_humidity'], row['min_range_rainfall'], row['max_range_rainfall']))
    conn.commit()
    cursor.close()

df_period.to_csv('result/predicted_tanaman.csv', index=False)